In [ ]:
import pandas as pd
import altair as alt
from vega_datasets import data as vega_data

In [ ]:
df = pd.read_csv('postings.csv')

print(f"Loaded {len(df)} job postings")
print(f"Columns: {list(df.columns)}")

def prepare_data(df):
    """Prepare and clean the job postings dataset"""
    data = df.copy()
    
    # Clean salary data
    if 'normalized_salary' in data.columns:
        data['salary'] = data['normalized_salary']
    else:
        data['salary'] = data[['min_salary', 'max_salary', 'med_salary']].mean(axis=1)
    
    # Remove outliers and missing values
    data = data.dropna(subset=['salary', 'formatted_experience_level'])
    data = data[(data['salary'] >= 20000) & (data['salary'] <= 500000)]
    
    # Clean experience level
    data['experience_level'] = data['formatted_experience_level'].fillna('Not Specified')
    
    # Remote flag
    data['remote'] = data['remote_allowed'].fillna(0).astype(bool)
    
    # Calculate engagement rate
    data['engagement_rate'] = (data['applies'] / (data['views'] + 1)) * 100
    data['engagement_rate'] = data['engagement_rate'].clip(0, 100)
    
    # Work type
    data['work_type'] = data['formatted_work_type'].fillna('Not Specified')
    
    return data

# Prepare the data
data = prepare_data(df)
print(f"\nPrepared data: {len(data)} job postings")

print("\n" + "="*80)
print("SPATIAL ANALYSIS: Examining geographic patterns")
print("="*80)

# Extract state from location column
data_spatial = data.copy()
data_spatial['state'] = data_spatial['location'].str.split(',').str[-1].str.strip()
data_spatial['state'] = data_spatial['state'].str.upper()

# Remove non-state values
invalid_values = ['UNITED STATES', 'US', 'USA', 'REMOTE', '']
data_spatial = data_spatial[~data_spatial['state'].isin(invalid_values)]

# Keep only valid 2-letter state codes
data_spatial = data_spatial[data_spatial['state'].str.len() == 2]

print(f"\nJobs with valid state codes: {len(data_spatial)}")
print(f"Unique states: {len(data_spatial['state'].unique())}")

# Aggregate statistics by state
state_stats = data_spatial.groupby('state').agg({
    'job_id': 'count',
    'salary': 'mean',
    'engagement_rate': 'mean',
    'remote': lambda x: (x == True).sum() / len(x) * 100
}).reset_index()

state_stats.columns = ['state', 'job_count', 'avg_salary', 'avg_engagement', 'pct_remote']

print(f"\nTop 10 states by job count:")
print(state_stats.nlargest(10, 'job_count')[['state', 'job_count', 'avg_salary']].to_string())

# Map state codes to FIPS IDs for choropleth
state_id_map = {
    'AL': 1, 'AK': 2, 'AZ': 4, 'AR': 5, 'CA': 6, 'CO': 8, 'CT': 9, 'DE': 10,
    'FL': 12, 'GA': 13, 'HI': 15, 'ID': 16, 'IL': 17, 'IN': 18, 'IA': 19,
    'KS': 20, 'KY': 21, 'LA': 22, 'ME': 23, 'MD': 24, 'MA': 25, 'MI': 26,
    'MN': 27, 'MS': 28, 'MO': 29, 'MT': 30, 'NE': 31, 'NV': 32, 'NH': 33,
    'NJ': 34, 'NM': 35, 'NY': 36, 'NC': 37, 'ND': 38, 'OH': 39, 'OK': 40,
    'OR': 41, 'PA': 42, 'RI': 44, 'SC': 45, 'SD': 46, 'TN': 47, 'TX': 48,
    'UT': 49, 'VT': 50, 'VA': 51, 'WA': 53, 'WV': 54, 'WI': 55, 'WY': 56,
    'DC': 11
}

state_stats['id'] = state_stats['state'].map(state_id_map)



In [ ]:
# Load US states geography
us_states = alt.topo_feature(vega_data.us_10m.url, 'states')

# Base map (background)
base_map = alt.Chart(us_states).mark_geoshape(
    fill='#f0f0f0',
    stroke='white',
    strokeWidth=1
).project('albersUsa')

# --- MAP 1: Average Salary ---
salary_choropleth = alt.Chart(us_states).mark_geoshape(
    stroke='white',
    strokeWidth=1
).encode(
    color=alt.Color('avg_salary:Q',
                    scale=alt.Scale(scheme='blues', domain=[40000, 150000]),
                    legend=alt.Legend(title='Average Salary', format='$,.0f')),
    tooltip=[
        alt.Tooltip('state:N', title='State'),
        alt.Tooltip('job_count:Q', title='Jobs', format=','),
        alt.Tooltip('avg_salary:Q', title='Avg Salary', format='$,.0f'),
        alt.Tooltip('pct_remote:Q', title='% Remote', format='.1f')
    ]
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(state_stats, 'id', ['state', 'job_count', 'avg_salary', 'pct_remote'])
).project('albersUsa')

map_salary = (base_map + salary_choropleth).properties(
    width=900,
    height=500,
    title={
        'text': 'Average Salary by State',
        'subtitle': 'Geographic distribution of job salaries across the United States'
    }
)
map_salary